In [ ]:
cd drive/MyDrive/JAIST/Research/RL/movie-lens-simulator

In [ ]:
!make style
!make setup

In [ ]:
from src.data.movielens1M import get_MovieLens1M
from src.dataloader.movielens import MovieLensDataset
from src.dataloader.dataloader import get_MovieLensDataloaders
from src.model.embedding import Simulator
from src.trainer.embedding_trainer import Embedding_Trainer

from configs.config import CFG_DICT
from src.utils.file_handler import File_Handler

file_handler = File_Handler()
logger = file_handler.make_logger_file()

In [ ]:
device = 'cuda'

raw_datasets, processed_datasets = get_MovieLens1M(CFG_DICT['DATASET']['PATH'])

In [ ]:
user_simulator = Simulator(emb_size=CFG_DICT['USER_SIMULATOR']['DIM_EMB']).to(device)

user_simlator_datas = get_MovieLensDataloaders(
    model_type = 'USER_SIMULATOR',
    processed_data = processed_datasets
)

user_simulator_trainer = Embedding_Trainer(
    model_type = 'USER_SIMULATOR',
    model = user_simulator,
    loaders = user_simlator_datas,
    device = device
    )

user_simulator = user_simulator_trainer.train()
file_handler.save(user_simulator.state_dict(), 'user_simulator.pkl')

In [ ]:
initial_recommender = Simulator(emb_size=CFG_DICT['INITIAL_RECOMMENDER']['DIM_EMB']).to(device)

initial_recommender_datas = get_MovieLensDataloaders(
    model_type = 'INITIAL_RECOMMENDER',
    processed_data = processed_datasets
)

initial_recommender_trainer = Embedding_Trainer(
    model_type = 'INITIAL_RECOMMENDER',
    model = initial_recommender,
    loaders = initial_recommender_datas,
    device = device
    )

initial_recommender = initial_recommender_trainer.train()
file_handler.save(initial_recommender.state_dict(), 'initial_recommender.pkl')

In [ ]:
ratings, users, movies, history = processed_datasets

In [ ]:
import numpy as np
import torch



ratings, users, movies, history = processed_datasets


for user in np.arange(len(users)):
    user_feat = np.tile(users.iloc[user, :].to_numpy(), (len(movies), 1))
    user_feat = torch.from_numpy(user_feat).to(device)
    user_feat = torch.unsqueeze(user_feat, 1)


    movie_feat = torch.from_numpy(movies.to_numpy()).to(device)
    movie_feat = torch.unsqueeze(movie_feat, 1)

    history_feat = np.tile(history.iloc[user, :].to_numpy(), (len(movies), 1))
    history_feat = torch.from_numpy(history_feat).to(device)
    history_feat = torch.unsqueeze(history_feat, 1)

    break

sigmoid = nn.Sigmoid()

user_simulator.eval()
gt = sigmoid(user_simulator(user_feat, movie_feat, history_feat)).detach().cpu().numpy()

initial_recommender.eval()
pred = sigmoid(initial_recommender(user_feat, movie_feat, history_feat)).detach().cpu().numpy()

def _rank(arr: np.array):
    return np.argsort(np.argsort(pred))

In [ ]:
topK = 10

topK_idx = _rank(pred) < topK

topk_gt = (pred[topK_idx] > 0.5).astype(int)
topk_pred = (pred[topK_idx] > 0.5).astype(int)
topk_prob = pred[topK_idx]
